## Basic Numpy Math

Key points:
 - Addition
 - Multiplication of signle row or column vectors

In [2]:
import numpy as np

## Creating NumPy Arrys

In [3]:
# access shape
def print_shape(a, name):
    if len(a.shape) == 1:
        print(f"{name}: flat/1D vector: {a.shape}, ndim:{a.ndim}")
    elif len(a.shape) == 2:
        r = a.shape[0]
        c = a.shape[1]
        print(f"{name}: 2D matrix {a.shape}: {r} rows, {c} columns, ndim: {a.ndim}")
    else:
        print(f"{name}: High dimensional array: {a.shape}, ndim: {a.ndim}")

In [4]:
a1 = np.array([1, 2, 3])  # 1D array, not strictly a row or column
a2 = np.array([[4], [5], [6]]) # 3 rows, 1 column (3,1)
a3 = np.array([[-4], [-5], [-6]]) # 3 rows, 1 column
a4 = np.array([[1,2,3]]) # 1 row, 3 columns

for i, a in enumerate([a1, a2, a3, a4]):
    print_shape(a, f"a{i}")

a0: flat/1D vector: (3,), ndim:1
a1: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2
a2: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2
a3: 2D matrix (1, 3): 1 rows, 3 columns, ndim: 2


__Notes:__:

1. a1 (3,)	 is a flat vector, used most commonly in NumPy.
2. a1 (1, 3) is a 2D row vector, which is useful when performing matrix operations (e.g., dot product).

__Best Practice:__
1. Use (3,) if you’re working with plain vectors or broadcasting.
2. Use (1, 3) if you need to preserve row/column shape in linear algebra.

__reshape():__
1. It **never modifies in-place** — you must reassign: a = a.reshape(...) if you want to change a.
2. usually returns a view, so it’s memory-efficient.
3. If not possible (due to layout), it returns a copy.

In [9]:
a1_reshaped = a1.reshape(-1, 1)
print(f"Shape of a1_reshaped: {a1_reshaped.shape}, ndim: {a1_reshaped.ndim}")
a1_reshaped[1,0] = 100
print(a1_reshaped)
a1_reshaped2 = a1.reshape(1, -1)
print(
    f"Shape of a1_reshaped2: {a1_reshaped2.shape}, ndim: {a1_reshaped2.ndim}")
print(a1_reshaped2)

Shape of a1_reshaped: (3, 1), ndim: 2
[[  1]
 [100]
 [  3]]
Shape of a1_reshaped2: (1, 3), ndim: 2
[[  1 100   3]]


In [10]:
print(a1)
print(f"a1: {a1.shape}, ndim: {a1.ndim}")
print(f"a1_reshaped: {a1_reshaped.shape}, ndim: {a1_reshaped.ndim}")
print(f"a1_reshaped2: {a1_reshaped2.shape}, ndim: {a1_reshaped2.ndim}")


[  1 100   3]
a1: (3,), ndim: 1
a1_reshaped: (3, 1), ndim: 2
a1_reshaped2: (1, 3), ndim: 2


In [11]:
print_shape(a1, "a1")
print_shape(a2, "a2")
print_shape(a3, "a3")
print_shape(a1_reshaped, "a1_reshaped")
print_shape(a1_reshaped2, "a1_reshaped2")


a1: flat/1D vector: (3,), ndim:1
a2: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2
a3: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2
a1_reshaped: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2
a1_reshaped2: 2D matrix (1, 3): 1 rows, 3 columns, ndim: 2


In [12]:
a2_ravel = a2.ravel()
print_shape(a2_ravel, "a2_ravel")

a2_ravel: flat/1D vector: (3,), ndim:1


## Broadcasting

See [NumPy documentatin](https://numpy.org/doc/stable/user/basics.broadcasting.html).
 - Move Right to left
 - Dimensions are compation when: they are equal or either one is 1

A set of arrays is called “broadcastable” to the same shape if the above rules produce a valid result.



In [52]:
# Create 10 straight points
np.random.seed(1024)
x, y = np.linspace(-10, 10, 21), np.random.randint(-5, 5, 21)
# Arrange then as [[x1,y1], [x2,y2], ...]
line = np.column_stack([x, y])
centroids = np.random.randint(-10, 10, size=(3, 2))
line[:2], centroids

(array([[-10.,  -4.],
        [ -9.,  -4.]]),
 array([[-5,  9],
        [ 6,  3],
        [ 5,  2]]))

Line will be broadcast to:

```text
[
  [ [point_0] ],  → will be broadcast with 3 centroids → 3 differences
  [ [point_1] ],  → will be broadcast with 3 centroids → 3 differences
  ...
  [ [point_20] ]
]
```


And the centroids will be broadcast to:
```text
[
  [
    [centroid_0],
    [centroid_1],
    [centroid_2]
  ]
]
```

Teh runtime will be:
```python
for i in range(21):     # loop over points
    for j in range(3):  # loop over centroids
        distances[i, j] = line[i] - centroids[j]
```

In [49]:
# line: (21, 2), centroids: (3, 2)
# line[:, np.newaxis, :]: (21, 1, 2)
# centroids: (3, 2) =>    (1, 3, 2)
# distances: (21, 3) as a result of norm: √((x1 - cx)**2 + (y1 - cy)**2)
distances = np.linalg.norm(line[:, np.newaxis, :] - centroids, axis=2) # Or axis=-1
distances[:2]

array([[14.03566885,  8.60232527,  5.38516481],
       [13.34166406,  6.40312424,  5.65685425]])

Explaining Broadcasting for line coordinates. It is just for explanation. It is not how actually it will work out

Lines `21,2 => 21,1,2 ==> 21,3,2`

```python
[ 
  [[-10.,  -4.], [-10.,  -4.], [-10.,  -4.]],
  [[ -9.,  -4.], [ -9.,  -4.], [ -9.,  -4.]],
  [[ -8.,   0.], [ -8.,   0.], [ -8.,   0.]],
  [[ -7.,   4.], [ -7.,   4.], [ -7.,   4.]],
  [[ -6.,  -4.], [ -6.,  -4.], [ -6.,  -4.]]
]
```

Centrioids `3,2 => (1,3,2) => (21, 3, 2)`:

```python
[ 
    [[-5,  9], [ 6,  3], [ 5,  2]], # 21 times, 
    [[-5,  9], [ 6,  3], [ 5,  2]],
    [[-5,  9], [ 6,  3], [ 5,  2]],
    [[-5,  9], [ 6,  3], [ 5,  2]],
]
```

In [54]:
centroids

array([[-5,  9],
       [ 6,  3],
       [ 5,  2]])

## Axes

Use it on NumPy 
axis = 0, along rows (Add vertically)
axis = 1, along columns

In [40]:
a1, a2

(array([1, 2, 3]),
 array([[4],
        [5],
        [6]]))

In [46]:
# a1: (3,), a2: (3,1)
print_shape(a1, "a1")
print_shape(a2, "a2")
a1_sum_a2 = a1 + a2

a1: flat/1D vector: (3,), ndim:1
a2: 2D matrix (3, 1): 3 rows, 1 columns, ndim: 2


In [44]:
A = np.array([[1,2, 3]])
B = np.array([10, 20, 30])
print_shape(A, "A")
print_shape(B, "B")
C = A + B
print_shape(C, "C")
print(C)

A: 2D matrix (1, 3): 1 rows, 3 columns, ndim: 2
B: flat/1D vector: (3,), ndim:1
C: 2D matrix (1, 3): 1 rows, 3 columns, ndim: 2
[[11 22 33]]


In [53]:
C = np.arange(12).reshape((3,4))
print(C)
col_sum = np.sum(C, axis=0)
print(col_sum)
print_shape(col_sum, "col_sum")
row_sum = np.sum(C, axis=1)
print(row_sum)
print_shape(row_sum, "row_shape")

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[12 15 18 21]
col_sum: flat/1D vector: (4,), ndim:1
[ 6 22 38]
row_shape: flat/1D vector: (3,), ndim:1
